# Esercizio 4 - Segmentation

## Funzionamento

Andiamo a calcolare la frequenza delle parole più utilizzate nel documento. La somma delle frequenze delle parole più utilizzate rappresentarà il
punteggio assegnato ad ogni segmento. lo scopo dell'algoritmo è trovare i tagli in modo da massimizzare la somma del punteggio.
Nella prima implementazione l'algoritmo ha bisogno del numero di tagli in ingresso.

- **COOCCORENCE** : Indica la frequenza delle parole più utilizzate nel segmento

## Dataset

Il dataset è costituito da pezzi dei paragrafi di wikipedia di 3 argomenti diversi, in questo caso i 3 argomenti sono:
- Gorillas
- Quantum Computing
- Astronomy

I tagli corretti sono alla riga 59/60 e alla riga 102/103.

### Imports

In [7]:
from nltk.corpus import stopwords
from collections import Counter
from gensim.test.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import random

### Methods

In [155]:
def remove_stop_words_ita(phrase):
    stop_words = stopwords.words('italian')
    phrase = phrase.split()
    phrase = [word for word in phrase if word not in stop_words]
    return phrase

def get_text_from_file(path):
    file = []
    stop_words = set(stopwords.words('english'))
    with open (path, 'r') as f:
        for row in f:
            filtered_s = [w for w in word_tokenize(row) if not w.lower() in stop_words]
            file.append(simple_preprocess(str(filtered_s), deacc=True))
    f.close()
    return file

def cooccurrence(text, n_most_words):
    '''
    Calculates the cooccurrence value of a sequence of words. 
    This value correspond to the sum of the occurrences of the n most frequently used words in the word list.
    '''
    score = 0
    c = Counter()
    most_common = []
    for row in text:
        c.update(row)
        
    most_common = c.most_common(n_most_words)
    # print(most_common)
    
    for el in most_common:
        score = score + el[1]
        
    return score

def extract_segment(file, start, end):
    '''
    Given the first and the last line, extract the segment.
    '''
    segment = []
    for i in range(start, end):
        segment.append(file[i])
    return segment

def get_cuts(ntopic):
    '''
    Generate a list of random value between 1 and the number of lines in the documents,
    that correspond to the cuts of the documents. The first value is 0 and the last is
    the numebers of lines in the documents.
    '''
    cut = [0] * ntopic
    while not(all(cut[i] < cut[i+1] for i in range(len(cut) - 1))):
        for k in range(1, ntopic): # generate the cuts
            cut[k] = random.randint(1, num_lines-1)
    cut.append(num_lines)
        
    return cut

### Data Processing

Dopo aver ripulito il testo, rimuovendo le stopword, organizziamo le parole in una lista costituita da liste di parole, 
ogni lista contiene le parole tokenizzate di una linea del testo. 


In [4]:
file = get_text_from_file('../res/segmentation_eng.txt')
c = Counter()
num_lines = sum(1 for line in open('../res/segmentation_eng.txt')) # Number of lines in the file

for row in file:
    c.update(row)
    
print(c.most_common(30))

[('quantum', 63), ('gorilla', 34), ('gorillas', 31), ('astronomy', 25), ('also', 17), ('classical', 17), ('algorithm', 16), ('stars', 16), ('silverback', 15), ('astronomical', 15), ('ray', 15), ('females', 14), ('computers', 14), ('computer', 14), ('wavelengths', 14), ('males', 13), ('known', 13), ('model', 13), ('algorithms', 13), ('early', 13), ('earth', 13), ('light', 13), ('infrared', 13), ('male', 12), ('problems', 12), ('planets', 12), ('western', 11), ('ft', 11), ('may', 11), ('made', 11)]


### Testing Methods

In [5]:
scores = []
cut = [0 , 3, 45, num_lines]
# Extract the segments
for i in range(len(cut)-1):
    print(cut[i])
    text = extract_segment(file, cut[i], cut[i+1])
    scores.append(cooccurrence(text))
    
print(scores)

0
3
45
[7, 64, 105]


### Main Algorithm

### Parameters - no more used

In [189]:
# x , y, max = 0, 0, 0 # x = cut1, y = cut2, max = max cooccurrence value
# max_scores = [(0, 0), (0, 0), (0, 0)] # Used to store the value of a segment with the highest cooccurrence value
# max_cut = [] # Used to store the cuts of the segments with the highest cooccurrence value
# scores = [0, 0, 0] # Inizialize the array to store the cooccurrence value of each segment

### Algorithm

In [225]:
def max_scores(file, ntopic, n_most_words): # n_most_words = number of most used words, used in cooccurrence()
    scores = [0]*ntopic
    sum_scores, max = 0, 0
    max_cut = []
    
    for f in range(2000): 
        # First generate the random cuts
        cut = get_cuts(ntopic)
        
        # Extract the segments by the cuts and calculate the cooccurrence value for each segment
        # based on the random cuts
        for i in range(len(cut)-1):
            text = extract_segment(file, cut[i], cut[i+1])
            scores[i] = cooccurrence(text, n_most_words)
        
        # Evaluate the result and store the best result
        sum_scores = sum(scores)
        if(sum_scores > max):
            max = sum_scores
            max_cut = cut
            
    return max_cut, max

In [228]:
n_most_words = 3
n_topics = 3
n_iteration = 2000 # Number of iteration to generate the best result - higher value = more time and better result

res = max_scores(file, n_topics, n_most_words)

print(f'\n Best cut at lines \n{res[0]}\n the max sum is \n{res[1]}\n')



 Best cut at lines 
[0, 54, 102, 181]
 the max sum is 
229



# Just a Try - Segmentation without number of topics
## Non funziona e in ogni caso impiega troppo tempo ad eseguire

In [139]:
max = 0
max_cut = []
max_n_topic = 0

for f in range(2):
    ntopic = random.randint(2, 10)
    res = max_scores(file, ntopic)
    
    if(max < res[1]):
        max = res[1]
        max_cut = res[0]
        max_n_topic = ntopic
        
print(f'max is {max} at {max_cut} with {max_n_topic} topics')


max is 300 at [0, 17, 54, 81, 100, 120, 158, 173, 181] with 8 topics
